In [4]:
import json
import pandas as pd
import numpy as np

# 融合数据
def load(modal, dataset='gaia', n=20):
    data = {}
    for i in range(n):
        with open(f'{dataset}/{modal}/{dataset}_{modal}_{i}.json', 'r') as f:
            temp = json.load(f)
        for key in temp:
            data[key] = temp[key]
    return data

def time_per_case(cases, dataset='gaia', n=20):
    time_total = 0
    for modal in ['trace', 'log', 'metric']:
        for i in range(n):
            with open(f'{dataset}/{modal}/time_used_{i}', 'r') as f:
                time_total += float(f.readlines()[0].split('\n')[0])
    return time_total / len(cases)            


In [12]:
# gaia
label_path = '/Users/fengxiaoyu/Desktop/PDiagnose/gaia_resplit.csv' # 故障case的路径
demo_labels = pd.read_csv(label_path, index_col=0)

services = ['dbservice1', 'dbservice2', 'logservice1', 'logservice2', 'mobservice1', 'mobservice2', 
           'redisservice1', 'redisservice2', 'webservice1', 'webservice2']

trace_total = load('trace', 'gaia', 20)
log_total = load('log', 'gaia', 20)
metric_total = load('metric', 'gaia', 20)

def evaluation(ranks, cases):
    tops = np.zeros(5)
    for case_id, case in cases.iterrows():
        for i in range(5):
            if ranks[case_id][i][0] == case['instance']:
                tops[i: ] += 1
                break
    return tops / len(cases)

# 参数
alpha = 5

ranks = {}
for case_id, case in demo_labels.loc[160: 260].iterrows(): 
# for case_id, case in demo_labels.loc[963: 1024].iterrows(): 
    rank = []
    # suspicious_set = set(trace_total[str(case_id)]) or set(log_total[str(case_id)]) # 可疑微服务集
    suspicious_set = set(trace_total[str(case_id)]) # rm log
    # suspicious_set = set(log_total[str(case_id)]) # rm trace
    for service in services:
        vote = 0
        if service in suspicious_set:
            vote += alpha
        # vote += len([_ for item in metric_total[str(case_id)] if item[1]==service])
        rank.append((service, vote))
    rank = sorted(rank, key=lambda x: x[1], reverse=True)
    ranks[case_id] = rank
result_list = evaluation(ranks, demo_labels.loc[160: 260]).tolist()
result_list

[0.06930693069306931,
 0.10891089108910891,
 0.16831683168316833,
 0.2079207920792079,
 0.4752475247524752]

In [20]:
import json
import pandas as pd
import numpy as np

# 融合数据
def load(modal, dataset='gaia', n=20):
    data = {}
    for i in range(n):
        with open(f'{dataset}/{modal}/{dataset}_{modal}_{i}.json', 'r') as f:
            temp = json.load(f)
        for key in temp:
            data[key] = temp[key]
    return data

# 21aiops
label_path = '/Users/fengxiaoyu/Desktop/PDiagnose/aiops21_groundtruth.csv' # 故障case的路径
demo_labels = pd.read_csv(label_path, index_col=0)

services = 'IG01 IG02 MG01 MG02 Mysql01 Mysql02 Redis01 Redis02 Tomcat01 Tomcat02 Tomcat03 Tomcat04 apache01 apache02 dockerA1 dockerA2 dockerB1 dockerB2'
services = services.split()

def evaluation(ranks, cases):
    tops = np.zeros(5)
    for case_id, case in cases.iterrows():
        for i in range(5):
            if ranks[case_id][i][0] == case['instance']:
                tops[i: ] += 1
                break
    return tops / len(cases)

trace_total = load('trace', '21aiops', 10)
log_total = load('log', '21aiops', 10)
metric_total = load('metric', '21aiops', 10)

# 参数
alpha = 5

ranks = {}
for case_id, case in demo_labels[demo_labels['data_type']=='test'].iterrows():
    rank = []
    # suspicious_set = set(trace_total[str(case_id)]) or set(log_total[str(case_id)]) # 可疑微服务集
    # # suspicious_set = set(trace_total[str(case_id)]) # rm log
    # suspicious_set = set(log_total[str(case_id)]) # rm trace
    for service in services:
        vote = 0
        # if service in suspicious_set:
        #     vote += alpha
        vote += len([_ for item in metric_total[str(case_id)] if item[1]==service])
        rank.append((service, vote))
    rank = sorted(rank, key=lambda x: x[1], reverse=True)
    ranks[case_id] = rank

evaluation(ranks, demo_labels[demo_labels['data_type']=='test'])

array([0.34177215, 0.53164557, 0.58227848, 0.67088608, 0.72151899])

In [8]:
evaluation(ranks, demo_labels[demo_labels['data_type']=='test'])

array([0.05063291, 0.17721519, 0.3164557 , 0.39240506, 0.48101266])

In [9]:
ranks

{75: [('IG01', 5),
  ('MG01', 5),
  ('MG02', 5),
  ('Tomcat01', 5),
  ('Tomcat02', 5),
  ('Tomcat03', 5),
  ('Tomcat04', 5),
  ('dockerA1', 5),
  ('dockerA2', 5),
  ('dockerB1', 5),
  ('dockerB2', 5),
  ('IG02', 0),
  ('Mysql01', 0),
  ('Mysql02', 0),
  ('Redis01', 0),
  ('Redis02', 0),
  ('apache01', 0),
  ('apache02', 0)],
 77: [('IG01', 5),
  ('IG02', 5),
  ('MG01', 5),
  ('MG02', 5),
  ('Tomcat01', 5),
  ('Tomcat02', 5),
  ('Tomcat03', 5),
  ('Tomcat04', 5),
  ('dockerA1', 5),
  ('dockerA2', 5),
  ('dockerB1', 5),
  ('dockerB2', 5),
  ('Mysql01', 0),
  ('Mysql02', 0),
  ('Redis01', 0),
  ('Redis02', 0),
  ('apache01', 0),
  ('apache02', 0)],
 78: [('IG01', 5),
  ('IG02', 5),
  ('MG01', 5),
  ('MG02', 5),
  ('Tomcat01', 5),
  ('Tomcat02', 5),
  ('Tomcat03', 5),
  ('Tomcat04', 5),
  ('dockerA1', 5),
  ('dockerA2', 5),
  ('dockerB1', 5),
  ('dockerB2', 5),
  ('Mysql01', 0),
  ('Mysql02', 0),
  ('Redis01', 0),
  ('Redis02', 0),
  ('apache01', 0),
  ('apache02', 0)],
 79: [('IG01', 5),


In [10]:
time_per_case(demo_labels[demo_labels['data_type']=='test'], '21aiops', 12)

FileNotFoundError: [Errno 2] No such file or directory: '21aiops/trace/time_used_10'

In [13]:
import json
import pandas as pd
import numpy as np
# 22aiops
label_path = '/Users/fengxiaoyu/Desktop/PDiagnose/22AIOps_run_table.csv' # 故障case的路径
demo_labels = pd.read_csv(label_path, index_col=0)
unique_cmdb_ids_list = ['emailservice-0', 'productcatalogservice-0', 'adservice-0', 'checkoutservice-0', 'shippingservice-0', 'checkoutservice-2', 'emailservice-1', 'cartservice-0', 'recommendationservice-0',
 'paymentservice2-0', 'adservice-2', 'currencyservice-1', 'cartservice-1', 'frontend-0', 'frontend-2', 'currencyservice2-0', 'recommendationservice2-0', 'shippingservice-1', 'frontend2-0', 'emailservice2-0',
 'shippingservice-2', 'frontend-1', 'paymentservice-0', 'paymentservice-1', 'cartservice2-0', 'productcatalogservice-1', 'adservice2-0', 'recommendationservice-1', 'checkoutservice-1', 'paymentservice-2',
 'cartservice-2', 'productcatalogservice-2', 'emailservice-2', 'shippingservice2-0', 'recommendationservice-2', 'currencyservice-2', 'currencyservice-0', 'adservice-1', 'productcatalogservice2-0', 'checkoutservice2-0']

# 融合数据
def load(modal, dataset='gaia', n=20):
    data = {}
    for i in range(n):
        with open(f'{dataset}/{modal}/{dataset}_{modal}_{i}.json', 'r') as f:
            temp = json.load(f)
        for key in temp:
            data[key] = temp[key]
    return data

def evaluation(ranks, cases):
    tops = np.zeros(5)
    for case_id, case in cases.iterrows():
        for i in range(5):
            if ranks[case_id][i][0] == case['cmdb_id']:
                tops[i: ] += 1
                break
    return tops / len(cases)

trace_total = load('trace', '22aiops', 1)
log_total = load('log', '22aiops', 1)
metric_total = load('metric', '22aiops', 12)

# 参数
alpha = 5

ranks = {}
for case_id, case in demo_labels[(demo_labels['type'] == 'test') & (demo_labels['level'] == 'pod')].iterrows():
    rank = []
    suspicious_set = set(trace_total[str(case_id)]) or set(log_total[str(case_id)]) # 可疑微服务集
    # suspicious_set =  set(trace_total[str(case_id)]) # 可疑微服务集
    for service in unique_cmdb_ids_list:
        vote = 0
        if service in suspicious_set:
            vote += alpha
        vote += len([_ for item in metric_total[str(case_id)] if item[1]==service])
        rank.append((service, vote))
    rank = sorted(rank, key=lambda x: x[1], reverse=True)
    ranks[case_id] = rank
evaluation(ranks, demo_labels[(demo_labels['type'] == 'test') & (demo_labels['level'] == 'pod')])

FileNotFoundError: [Errno 2] No such file or directory: '22aiops/metric/22aiops_metric_0.json'

In [ ]:
ranks

In [28]:
import json
import pandas as pd
import numpy as np
# platform
label_path = '/Users/fengxiaoyu/Desktop/PDiagnose/run_table.csv' # 故障case的路径
demo_labels = pd.read_csv(label_path, index_col=0)
unique_cmdb_ids_list=['cartservice','checkoutservice','currencyservice','emailservice','frontend','paymentservice','productcatalogservice','recommendationservice','shippingservice']


# 融合数据
def load(modal, dataset='gaia', n=20):
    data = {}
    for i in range(n):
        with open(f'{dataset}/{modal}/{dataset}_{modal}_{i}.json', 'r') as f:
            temp = json.load(f)
        for key in temp:
            data[key] = temp[key]
    return data

def evaluation(ranks, cases):
    tops = np.zeros(5)
    for case_id, case in cases.iterrows():
        for i in range(5):
            if ranks[case_id][i][0] == case['service']:
                tops[i: ] += 1
                break
    return tops / len(cases)

trace_total = load('trace', 'platform', 1)
log_total = load('log', 'platform', 1)
metric_total = load('metric', 'platform', 10)

# 参数
alpha = 5

ranks = {}
for case_id, case in demo_labels[(demo_labels['data_type'] == 'test')].iterrows():
    rank = []
    # suspicious_set = set(trace_total[str(case_id)]) or set(log_total[str(case_id)]) # 可疑微服务集
    suspicious_set =  set(log_total[str(case_id)]) # 可疑微服务集
    # suspicious_set =  set(trace_total[str(case_id)]) # 可疑微服务集
    for service in unique_cmdb_ids_list:
        vote = 0
        if service in suspicious_set:
            vote += alpha
        vote += len([_ for item in metric_total[str(case_id)] if item[1]==service])
        rank.append((service, vote))
    rank = sorted(rank, key=lambda x: x[1], reverse=True)
    ranks[case_id] = rank
evaluation(ranks, demo_labels[(demo_labels['data_type'] == 'test')])

array([0.24050633, 0.48101266, 0.58227848, 0.60759494, 0.60759494])

In [ ]:
ranks